# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
cities_nhl = cities.copy()
    
# Remove the 'note[]'

cities_nhl['NHL'] = cities_nhl['NHL'].apply(lambda x: re.sub( '\[.*\]', '', x))

# Get the values distinct of '-' and ''

cities_nhl = cities_nhl.loc[~(cities_nhl['NHL'] == '—') & ~(cities_nhl['NHL'] == '')]

# Get the usefull columns and rename for convenience

cities_nhl = cities_nhl[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

# Clean the team field and get only the name of each team

nhl_df['team'] = nhl_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])

nhl_df = nhl_df.loc[nhl_df['year'] == 2018]

nhl_df = nhl_df[nhl_df['team'] != 'Division']

'''

TO CHECK FOR CITIES WITH MORE THAN ONE TEAM OR WRITING DIFFERENCES BETWEEN THE TWO DATAFRAMES AND MAKE THE CORRECTIONS

print(nhl_df['team'].values)

print(cities_nba['NHL'].values)

for i in nhl_df['team']:
    
    if i not in cities_nba['NHL'].values:
        print(i)


'''

cities_nhl["NHL"] = cities_nhl["NHL"].replace({'Rangers Islanders Devils': 'Rangers-Islanders-Devils',
                                           'Kings Ducks':'Kings-Ducks'})

cities_nhl["NHL"] = cities_nhl["NHL"].apply(lambda x: x.split('-'))

cities_nhl = cities_nhl.explode('NHL')

renameDict  = { 'Knights': 'Golden Knights',
                  'Leafs': 'Maple Leafs',
                  'Jackets': 'Blue Jackets',
                  'Wings': 'Red Wings'}

nhl_df['team'] = nhl_df['team'].apply(lambda x: renameDict[x] if x in renameDict else x)


df_nhl = pd.merge( nhl_df, cities_nhl, left_on = 'team', right_on = 'NHL')


df_nhl = df_nhl[['Metropolitan area', 'Population', 'team', 'W', 'L']]

cols =  ['Population', 'W', 'L']

df_nhl[cols] = df_nhl[cols].astype(int)

df_nhl = df_nhl.groupby('Metropolitan area').sum().reset_index()

 
df_nhl.loc[df_nhl['Metropolitan area'] == 'New York City', 'Population'] = 20153634

df_nhl.loc[df_nhl['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

df_nhl['W/L'] = df_nhl['W'] / (df_nhl['W'] + df_nhl['L'])

df_nhl['Population'] = df_nhl['Population'].astype(float)

df_nhl['W/L'] = df_nhl['W/L'].astype(float)

population_by_region = df_nhl['Population']

win_loss_by_region = df_nhl['W/L']

assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
result = stats.pearsonr(population_by_region, win_loss_by_region)[0]

result


0.012308996455744264

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



def nhl_correlation(): 

    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities_nhl = cities.copy()

    # Remove the 'note[]'

    cities_nhl['NHL'] = cities_nhl['NHL'].apply(lambda x: re.sub( '\[.*\]', '', x))

    # Get the values distinct of '-' and ''

    cities_nhl = cities_nhl.loc[~(cities_nhl['NHL'] == '—') & ~(cities_nhl['NHL'] == '')]

    # Get the usefull columns and rename for convenience

    cities_nhl = cities_nhl[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

    # Clean the team field and get only the name of each team

    nhl_df['team'] = nhl_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])

    nhl_df = nhl_df.loc[nhl_df['year'] == 2018]

    nhl_df = nhl_df[nhl_df['team'] != 'Division']

    cities_nhl["NHL"] = cities_nhl["NHL"].replace({'Rangers Islanders Devils': 'Rangers-Islanders-Devils',
                                               'Kings Ducks':'Kings-Ducks'})

    cities_nhl["NHL"] = cities_nhl["NHL"].apply(lambda x: x.split('-'))

    cities_nhl = cities_nhl.explode('NHL')

    renameDict  = { 'Knights': 'Golden Knights',
                      'Leafs': 'Maple Leafs',
                      'Jackets': 'Blue Jackets',
                      'Wings': 'Red Wings'}

    nhl_df['team'] = nhl_df['team'].apply(lambda x: renameDict[x] if x in renameDict else x)


    df_nhl = pd.merge( nhl_df, cities_nhl, left_on = 'team', right_on = 'NHL')


    df_nhl = df_nhl[['Metropolitan area', 'Population', 'team', 'W', 'L']]

    cols =  ['Population', 'W', 'L']

    df_nhl[cols] = df_nhl[cols].astype(int)

    df_nhl = df_nhl.groupby('Metropolitan area').sum().reset_index()


    df_nhl.loc[df_nhl['Metropolitan area'] == 'New York City', 'Population'] = 20153634

    df_nhl.loc[df_nhl['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

    df_nhl['W/L'] = df_nhl['W'] / (df_nhl['W'] + df_nhl['L'])
    
    

    population_by_region = df_nhl['Population']

    win_loss_by_region = df_nhl['W/L']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    result = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return result

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.